In [ ]:
from pprint import pprint
import requests

# suppress warning: https://stackoverflow.com/a/44850849/9422455
requests.packages.urllib3.disable_warnings()

COOKIE = 'Cookie: SESSION=MDlmZjZhZTgtYjJhYy00MzBiLTgxNjctODQzMGQxM2IxNzc2; SID=066083; token=09ff6ae8-b2ac-430b-8167-8430d13b1776; Ecp_ClientId=2211228160403205403; Ecp_IpLoginFail=21122849.80.63.68; Ecp_lout=1'
OTHERS = '''Accept-Encoding: gzip, deflate, br
Accept-Language: zh-CN,zh;q=0.9
Connection: keep-alive
DNT: 1
Host: dict.cnki.net
Referer: https://dict.cnki.net/index
sec-ch-ua: " Not A;Brand";v="99", "Chromium";v="96", "Google Chrome";v="96"
sec-ch-ua-mobile: ?0
sec-ch-ua-platform: "macOS"
Sec-Fetch-Dest: empty
Sec-Fetch-Mode: cors
Sec-Fetch-Site: same-origin
Token: 09ff6ae8-b2ac-430b-8167-8430d13b1776
Accept: application/json, text/plain, */*
Referer: https://dict.cnki.net/index
Token: 09ff6ae8-b2ac-430b-8167-8430d13b1776'''

headers_str = '''Token: 09ff6ae8-b2ac-430b-8167-8430d13b1776
User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36'''

headers_dict = dict(i.split(': ', 1) for i in headers_str.splitlines())


# s = requests.Session()
# s.headers = headers_dict

def fetch_get_trans(word: str, topic_code: str = None):
    url = 'https://dict.cnki.net/fyzs-front-api/translate/relatedliterature'
    params = {
        'words': word,
        'topicCode': topic_code,
        'pageNum': 1,
        'pageSize': 10,
    }
    try:
        res = requests.get(url, params, headers=headers_dict, verify=False)
        s = res.json()['data']
        pprint(s)
        return s
    except Exception as e:
        print(e)
        return None


def fetch_query_trans(word: str):
    url = 'https://dict.cnki.net/fyzs-front-api/translate/querytranslatedate'
    try:
        res = requests.post(url, json={'words': word}, verify=False, headers=headers_dict)
        #         pprint(res.json())
        return res.json()
    except Exception as e:
        print(e)
        return None


words = '封装'

res = fetch_query_trans(words)

if res is None:
    raise Exception("net failure")

'''
{'res': {'code': 200,
         'data': {'academicDicts': None,
                  'adictsVos': None,
                  'code': 200,
                  'dictsVos': None,
                  'ikWords': None,
                  'isInputVerificationCode': 0,
                  'mResult': '对不起，由于您检索过于频繁，数据查询失败！',
                  'machinetranslateFlag': 0,
                  'relatedSearchs': None,
                  'translateFlag': 0,
                  'translateMethod': 2,
                  'translateType': 1,
                  'words': '封装'},
         'msg': '操作成功'}}
'''
if res and '检索过于频繁' in res.get('data', {}).get('mResult', ""):
    raise Exception(res["data"]["mResult"])


rank_a = res['data']['dictsVos'][2]['adictsVo']['dicts'][words]
pprint(rank_a)

data_1 = res['data']['dictsVos']

In [ ]:
data_1

In [ ]:
from collections import defaultdict

# Do not use SortedDict, since it's not Chinese sort support, and seems only can sort by keys
# from sortedcontainers import SortedDict

data = defaultdict(dict)


def _safe_get(data, key, default):
    return data.get(key) or default


words = '封装'

key_info_list = data_1[0]['adictsVo']['dicts'][words][:10]

for item in data_1[1:]:
    topic = item.get('topicName', None)
    for (k, d) in [('adictsVo', {}), ('dicts', {}), (words, [])]:
        item = _safe_get(item, k, d)
    for item2 in item:
        key = item2['name']
        freq = item2['wordFreq']
        data[key][topic] = freq

for k, v in data.items():
    data[k] = sorted(v.items(), key=lambda x: -x[1])[:3]

items = [{"name": key_info['name'], "freq": key_info['wordFreq'], "details": data[key_info['name']]} for key_info in
         key_info_list]
items